In [2]:
import glob
from tqdm import tqdm_notebook as tqdm
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from PIL import Image
from torch import nn as nn
from torch import optim as optim
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import datasets as datasets
from torchvision import transforms as transforms
from torchvision.utils import make_grid
import torch.nn.functional as F


%matplotlib inline

In [56]:
class MyModel(nn.Module): 
    def __init__(self):                                               
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 3,out_channels = 16,kernel_size = 3,stride=1,padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2, return_indices=True)
        self.conv2 = nn.Conv2d(in_channels=16,out_channels=16,kernel_size=2,stride=1,padding=0)
        self.conv3 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=2,stride=1,padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=1,stride=1, return_indices=True)
        self.conv4 = nn.Conv2d(in_channels = 32,out_channels = 32,kernel_size = 7,stride=1,padding=0)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=2, return_indices=True)
        self.fc1 =  nn.Linear(32*7*8, 32*7*8)
        self.fc2 =  nn.Linear(32*7*8, 32*7*8)
        self.maxunpool1 = nn.MaxUnpool2d(kernel_size=2,stride=2)
        self.deconv1 = nn.ConvTranspose2d(in_channels=32,out_channels=32,kernel_size = 7,stride=1,padding=0)
        self.maxunpool2 = nn.MaxUnpool2d(kernel_size=1,stride=1)
        self.deconv2 = nn.ConvTranspose2d(in_channels=32,out_channels=16,kernel_size=2,stride=1,padding=1)
        self.deconv3 = nn.ConvTranspose2d(in_channels=16,out_channels=16,kernel_size=2,stride=1,padding=0)
        self.maxunpool3 = nn.MaxUnpool2d(kernel_size=2,stride=2)
        self.deconv4 = nn.ConvTranspose2d(in_channels = 16,out_channels = 3,kernel_size = 3,stride=1,padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x,ind1 = self.maxpool1(self.conv1(x))
        x = self.conv2(x)
        x, ind2 = self.maxpool2(self.conv3(x))
        x, ind3 = self.maxpool3(self.conv4(x))
        x = x.reshape(-1,32*7*8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = x.reshape(-1,32,7,8)
        x = self.maxunpool1(x,indices=ind3)
        x = self.deconv1(x)
        x = self.maxunpool2(x,indices=ind2)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.maxunpool3(x,indices=ind1,output_size = [4,16,40,45])
        x = self.deconv4(x)
        return x

In [1]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

alist=[
    "something1",
    "something12",
    "something17",
    "something2",
    "something25",
    "something29"]

alist.sort(key=natural_keys)
print(alist)

['something1', 'something2', 'something12', 'something17', 'something25', 'something29']


In [57]:
import cv2

class OFSDataset(Dataset):
    def __init__(self,transform):
        
        self.input_img = None
        self.label_img = None
        self.transform = None
        
        input_image_path=glob.glob("/home/saikrishna/Desktop/sem4/project/dataset/HS_slices(don't_open)/*.png")
        label_image_path=glob.glob("/home/saikrishna/Desktop/sem4/project/dataset/CE_slices(don't_open)/*.png")
        
        input_image_path.sort(key = natural_keys)
        label_image_path.sort(key = natural_keys)
        
        input_img = []
        label_img = []
        inp_list = []
        lab_list = []
        
        for i in range(1,1281):
            inp_im = cv2.imread(input_image_path[i])
            inp_list.append(inp_im)
            
            lab_im = cv2.imread(label_image_path[i])
            lab_list.append(lab_im)
            
        self.input_img = inp_list
        self.label_img = lab_list
    
    def __len__(self):
        return 1280

    def __getitem__(self, i):
        assert 0<=i<1281, f'i={i} corresponding to item number {i+1} exceeds size of the dataset that is {self.__len__()}'
        
        input_image = self.input_img[i]
        label_image = self.label_img[i]
        
        
#         print(self.transform)
        
        if self.transform is not None:
            input_image = self.transform(input_image)
            label_image = self.transform(label_image)
     
    # 45*40*4 = 7200
    # 45*40 = 1800
    
        else:
#             print("input_image.shape :",input_image.shape) #(45,40,4)
            input_image = input_image.reshape(40*45, 3)
            input_image = input_image.transpose(1, 0)
            input_image = input_image.reshape(3, 40, 45)            
            input_image = input_image/255.
            input_image = torch.from_numpy(input_image)
            input_image = input_image.to(dtype=torch.float32)
            
            label_image = label_image.reshape(40*45, 3)
            label_image = label_image.transpose(1, 0)
            label_image = label_image.reshape(3, 40, 45)            
            label_image = label_image/255.
            label_image = torch.from_numpy(label_image)
            label_image = label_image.to(dtype=torch.float32)
            
        return input_image,label_image

In [58]:
def train(model, loader, loss_criterion, optimizer, num_epochs):
    #(4,3,40,45)
#     model.train()
    for epoch in tqdm(range(num_epochs)): 
        for images, labels in loader: 
            if torch.cuda.is_available():
                images = images.to(device = 'cuda')
                labels = labels.to(device = 'cuda')
#             print("ok")
#             print(f"images.shape - {images.shape} | labels.shape - {labels.shape}")
            
            outputs = model(images) # do forward propagation
            loss = loss_criterion(outputs.squeeze(), labels.to(dtype = torch.float)) # compute loss
            optimizer.zero_grad() # make sure gradients of parameters are zeroed before backpropagation to avoid accumulation
            loss.backward() # do the backpropagation
        optimizer.step() # update the parameters
        
        if epoch%25 == 0:
            print(f'Training loss at epoch {epoch+1}:\t{loss.item()}')

In [59]:
def main():

    mymodel = MyModel()

    if torch.cuda.is_available():
        mymodel = mymodel.to(device='cuda')

    loss_criterion = nn.MSELoss()
    optimizer = optim.SGD(mymodel.parameters(), lr=0.008)

    train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.ToTensor(),
    ])

    dset_train = OFSDataset(transform = None)
    train_loader = DataLoader(dset_train, batch_size=100, shuffle=True, num_workers=4)
    
    train(mymodel, train_loader, loss_criterion, optimizer, 500)
    
    torch.save(
        {'epoch': 500,
         'model': mymodel.state_dict(),
         'optimizer': optimizer.state_dict(),
         'loss': loss_criterion,
        }, '500_sgd_008_100.pth')
    
#     test('new_new_chkpt_500_sgd_008_64.pth', test_loader, loss_criterion)


if __name__ == '__main__':
    main()

Training loss at epoch 1:	0.948051929473877


Traceback (most recent call last):
  File "/home/saikrishna/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/saikrishna/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/saikrishna/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/saikrishna/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/saikrishna/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/saikrishna/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/saikrishna/anaconda3/lib/python3.7/multiprocessing/connection.py", line 40

KeyboardInterrupt: 